# **Trabajo Práctico - Procesamiento del Lenguaje Natural**
# *Integrante: Lucas Demarré*
# *Año: 2024*

---

# **Tabla de contenidos**

1.   [**Introducción**](#)
2.   [**Librerías a Utilizar**](#)
3.   [**Variables Globales**](#)
4.   [**Definición de Funciones**](#)
        1.   [*Funciones para Creación de Fuentes de Información*](#)
        2.   [*Funciones para Manejo de Datos PDF*](#)
        3.   [*Funciones para Manejo de Datos CSV*](#)
        4.   [*Funciones para la creación y consulta de Base de Datos de Grafos*](#)
        5.   [*Funciones para Clasificación de Intenciones*](#)
        6.   [*Funciones para Generar Respuesta del Chatbot*](#)
5.   [**Creación de Fuentes de Información**](#)
6.   [**Carga y Limpieza de Datos**](#)
        1.   [*Documentos PDF*](#)
        2.   [*Datos Tabulares (CSV)*](#)
        3.   [*Recolección y Etiquetado de Preguntas para Clasificación de Intenciones*](#)
7.   [**Procesamiento de Datos para Base de Datos Chroma (PDFs)**](#)
        1.   [*División de los Documentos en Pequeñas Partes*](#)
        2.   [*Conversión a Embeddings y Almacenamiento en Chroma*](#)
8.   [**Creación de la Base de Datos de Grafos**](#)
9.   [**Entrenamiento del Clasificador de Intenciones**](#)
        1.   [*Cargamos el Modelo de Embeddings*](#)
        2.   [*División en Conjuntos de Entrenamiento y Prueba y Tokenización*](#)
        3.   [*Entrenamiento y Evaluación del Modelo Clasificador*](#)
        4.   [*Validación del Modelo Clasificador*](#)
10.   [**Integración y Flujo de Trabajo del Chatbot**](#)
        1.   [*Definimos la consulta a hacer*](#)
        2.   [*Clasificación de Preguntas de Usuario*](#)
        3.   [*Búsqueda en la Base de Datos Correspondiente y Generación del Prompt*](#)
        4.   [*Generación de Respuesta con el Chatbot*](#)
11.  [**Demo del Chatbot**](#)
12.  [**Informe sobre Agentes Inteligentes (LLM libres)**](#)
       1.    [*Estado del Arte de las Aplicaciones Actuales*](#)
       2.    [*Problemática a Solucionar con un Sistema Multiagente*](#)
       3.    [*Ejemplo de conversación*](#)
       4.    [*Fuentes de Información*](#)

# 1. **Introducción**

Este proyecto trata de el diseño y creación de un **ChatBot Experto** especializado en **Videojuegos**. Este ChatBot se construyó con la capacidad de responder consultas específicas sobre el tema, utilizando un vasto conocimiento extraído de diversos títulos. La información que abarca incluye *géneros*, *descripciones*, *desarrolladores*, *distribuidoras*, *requisitos de sistema* y *precios*, almacenada inicialmente en archivos PDF's y posteriormente transferida a bases de datos para su fácil acceso y consulta.

### ¿Qué es el **Chatbot**?
El **Chatbot** representa el conjunto de todos los pasos a detallar, empezando en la creación de base de datos, utilización de embeddings o no dependiendo de la base de datos creada y el modelo de Lenguaje de Gran Tamaño utilizado para la posterior consulta.

## Elección de tema:
El primer paso en el desarrollo de este ChatBot es seleccionte el tema de interés, que para este proyecto son los videojuegos. La información se extrae de múltiples fuentes y se organiza en un formato estructurado para su posterior uso.

## Creación de Bases de Datos:.
La creación de bases de datos adecuadas es esencial, ya que actúan como el fundamento sobre el cual el ChatBot basa sus respuestas. Este proceso implica la organización de la información en tres tipos principales de bases de datos:

1. **Base de Datos Vectorial:** Aquí se almacenan oraciones transformadas en vectores a través de un proce*so de emb*edding, permitiendo su representación en un espacio multidimensional. Esto facilita la identificación de similitudes y diferencias entre consultas y la información almacenada, optimizando la precisión en las respuestas del ChatBot.
   
2. **Base de Datos de Grafos:** Utiliza una estructura de grafos para almacenar información, donde los nodos representan datos o conceptos interconectados por aristas. Esta estructura permite una búsqueda eficiente y flexible a través de las conexiones, facilitando el acceso a información relevante basada en la relación entre diferentes nodos.
   
3. **Datos Tabulares:** Estos se presentan en un formato más sencillo, almacenados en archivos CSV que se pueden consultar de manera directa, similar a un diccionario, accediendo a la información mediante claves y columnas.

## Proceso Final que lleva a cabo el **Chatbot**:
Para interactuar efectivamente con el ChatBot y extraer la información necesaria de estas bases de datos, se aplican diversas técnicas de Procesamiento de Lenguaje Natural (NLP). Esto incluye desde la extracción de palabras clave hasta la conversión de consultas completas en vectores, dependiendo de la naturaleza de la base de datos consultada. Además, se implementan procesos de limpieza de datos para optimizar el contexto proporcionado al modelo de Lenguaje de Gran Tamaño (LLM), que es el encargado de interpretar las consultas y generar respuestas coherentes y precisas.

# 2. **Librerías a utilizar**

In [58]:
import os
import re
import csv
import time
import torch
import shutil
import chromadb
import pdfplumber
import numpy as np
import pandas as pd
import requests as rq
import networkx as nx
import tensorflow_text
import tensorflow_hub as hub
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup
from langdetect import detect
from fuzzywuzzy import process
from reportlab.lib.units import inch
from langchain.schema import Document
from IPython.display import clear_output
from reportlab.lib.pagesizes import letter
from pdfminer.high_level import extract_text
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores.chroma import Chroma
from transformers import BertTokenizer, BertModel
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from reportlab.lib.styles import getSampleStyleSheet
from llama_index.embeddings import LangchainEmbedding
from langchain.document_loaders import DirectoryLoader
from reportlab.platypus import SimpleDocTemplate, Paragraph
from sklearn.metrics import accuracy_score, classification_report
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings

import warnings
warnings.filterwarnings('ignore')

# 3. **Variables Globales**

In [2]:
# Establecemos nuestra KEY para acceder a la API de HuggingFace
API_KEY = 'TU-API-KEY'

# Establecemos rutas de acceso para acceder/guardar los archivos necesarios
CHROMA_PATH = 'chroma' # donde se va a guardar la base de datos 'chroma'
PDF_PATH = 'databases/vectorDB' # ruta de los archivos PDF
CSV_PATH = 'databases/tabularDB/Games Price.csv' # ruta del archivo CSV

# Creamos una lista de etiquetas llamada que contiene pares de valores para categorizar las consultas al LLM
labels = [(0, 'Consulta PDF'), (1, 'Consulta CSV'), (2, 'Consulta Grafos')]

# Establecemos el nombre del modelo de 'Embedding' que utilizaremos para convertir el texto en vectores
model_embedding_chatbot = 'https://tfhub.dev/google/universal-sentence-encoder-multilingual/3'

# Definimos la plantilla de texto que usaremos como 'prompt' para generar una respuesta del LLM más estructurada
PROMPT_TEMPLATE = '''
La información de contexto es la siguiente:
---------------------
{context}
---------------------
Dada la información de contexto anterior, y sin utilizar conocimiento previo, responde la siguiente pregunta.
Pregunta: {question}
Respuesta: '''

# 4. **Definición de Funciones**

## 4.1. *Funciones para la Creación de Fuentes de Información*

En el proyecto opté por generar información propia en lugar de depender de fuentes externas. Esta decisión se basa en experiencias previas con la API de **Steam**, una plataforma líder a nivel mundial en la distribución de videojuegos. A pesar de que **Steam** ofrece acceso público a una vasta cantidad de datos a través de su API, la elección de construir *fuentes de conocimientos personalizadas* surge de otras cuestiones:

- **Formato Personalizado:** Crear las propias fuentes de información permite adaptar el formato de los datos a las necesidades específicas del proyecto, facilitando su integración y manipulación en el diseño del **ChatBot**.
- **Experiencia Previa:** La familiaridad previa con la API de **Steam** ofrece una ventaja técnica, permitiendo una extracción de datos más eficiente y efectiva.

Sin embargo, la tarea de recopilar datos de **Steam** presenta sus propios desafíos, principalmente debido a la vastedad del catálogo disponible y las limitaciones de la API. **Steam** cuenta con más de 180.000 títulos, lo que hace inviable la extracción completa de datos en un marco de tiempo razonable. Por lo tanto, se estableció un límite práctico para extraer información de los primeros 10.000 títulos, lo cual ya de por sí tarda más de 6 horas. Aun así, a pesar de la limitación, se cumplen con creces los requisitos establecidos en el trabajo, tan solo el PDF de *Requirements* tiene 171 páginas.

In [3]:
''' Definimos una función para obtener los nombres de los juegos desde una API '''
def bring_game_names():
    api_url = 'https://api.steampowered.com/ISteamApps/GetAppList/v2/' # Especificamos la URL de la API de Steam
    response = rq.get(api_url) # Realizamos la solicitud a la API
    
    appids = [] # Inicializamos una lista para almacenar los IDs y nombres de los juegos
    
    # Verificamos si la solicitud fue exitosa
    if response.status_code == 200:
        data = response.json() # Convertimos la respuesta en formato JSON
        
        # Recorremos la lista de juegos y añadimos sus IDs y nombres a nuestra lista
        for i in data['applist']['apps']: appids.append([str(i['appid']), str(i['name'])])
    return appids

''' Definimos una función para limpiar texto que contiene etiquetas HTML '''
def clean_text(text):
    # Usamos BeautifulSoup para parsear el HTML
    soup = BeautifulSoup(text, 'html.parser')

    # Obtenemos solo el texto, sin etiquetas HTML
    text = soup.get_text()
    return text

''' Definimos una función para detectar el idioma de un texto '''
def detect_language(text):
    try: return detect(text) # Intentamos detectar el idioma y lo devolvemos
    except: return "Error en la detección" # En caso de error, devolvemos un mensaje de error

''' Definimos una función con cuatro parámetros para detectar si un valor se encuentra o no en el json '''
def get_value(data, key, default, transform=None):
    '''
    `data`: el diccionario de donde queremos obtener el valor.
    `key`: la clave que estamos buscando en el diccionario.
    `default`: el valor por defecto que se devuelve si la clave no se encuentra.
    `transform`: una función opcional para transformar el valor obtenido; por defecto es None. 
    '''
    
    # Se intenta obtener el valor asociado con 'key' en el diccionario 'data'.
    # Si 'key' no se encuentra en 'data', se devuelve 'default'.
    value = data.get(key, default)

    # Si 'transform' no es None, se aplica 'transform' a 'value'.
    # Si 'transform' es None, simplemente se devuelve 'value'.
    return transform(value) if transform else value
    
''' Definimos una función para obtener detalles de los juegos '''
def game_details(games):
    start = time.time() # Marcamos el tiempo de inicio
    
    # Especificamos la URL de la API para detalles de juegos individuales
    api_url_item = 'https://store.steampowered.com/api/appdetails?appids='
    games_seen = 0
    total_games = len(games)
    
    # Creamos documentos PDF para diferentes categorías de información
    game_descriptions = SimpleDocTemplate('databases/vectorDB/Games Description.pdf', pagesize=letter)
    game_requirements = SimpleDocTemplate('databases/vectorDB/Games Requirement.pdf', pagesize=letter)
    game_details = SimpleDocTemplate('databases/graphDB/Games Detail.pdf', pagesize=letter)
    
    # Inicializamos listas para almacenar contenido de los PDFs
    story_gd = []
    story_gr = []
    story_gde = []
    
    styles = getSampleStyleSheet() # Obtenemos un conjunto estándar de estilos para los PDFs
    output_filename = 'databases/tabularDB/Games Price.csv' # Especificamos el nombre del archivo CSV a crear
    
    # Abrimos el archivo CSV para escritura
    with open(output_filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['nombre', 'precio']) # Escribimos la fila de encabezado en el CSV

        # Iteramos sobre la lista de juegos mientras haya juegos disponibles
        while games:        
            games_seen += 1
            
            # Extramos el último elemento de la lista y guardamos el ID y el nombre del juego
            game = games.pop()
            game_id = game[0]
            game_name = game[1]
            
            # Realizamos una solicitud a la API para obtener detalles del juego
            response = rq.get(api_url_item + str(game_id) + '&l=spanish')
            
            try:
                if response.status_code == 200: 
                    temp = response.json()

                    if temp[str(game_id)]['success']:

                        # Verificamos que el juego tiene categorías
                        if temp.get(str(game_id), {}).get('data', {}).get('categories'):
                            # Guardamos las categorías en una lista
                            data = temp[str(game_id)]['data'] 
                            game_caterogies = data['categories']
                            categories_list = [category['id'] for category in game_caterogies]

                            # Verificamos que no sea una demo o una banda sonora
                            if 21 not in categories_list and 'Demo' not in game_name and 'Soundtrack' not in game_name:
                                recommended_req = 'No especificado'
                                minimun_req = 'No especificado'

                                if type(data.get('pc_requirements', {})) != list:
                                    recommended_req = get_value(data['pc_requirements'], 'recommended', 'No especificado', clean_text)
                                    minimun_req = get_value(data['pc_requirements'], 'minimum', 'No especificado', clean_text)

                                price = get_value(data, 'price_overview', 'Gratis', lambda x: x['final_formatted'] if isinstance(x, dict) else x)
                                developers = get_value(data, 'developers', 'No especificado', lambda x: x[0])
                                publishers = get_value(data, 'publishers', 'No especificado', lambda x: x[0])
                                short_description = get_value(data, 'short_description', 'No especificado', clean_text)

                                # Detectamos el idioma de las descripciones
                                language_short = detect_language(short_description)

                                # Si el idioma es español, añadimos la información a los documentos PDF y al CSV
                                if language_short == 'es':    
                                    story_gd.append(Paragraph(f'<b>Descripción corta de {game_name}</b>: {short_description}', styles['Normal']))
                                    story_gd.append(Paragraph('<br/><br/>', styles['Normal']))

                                    story_gr.append(Paragraph(f'<b>Requisitos mínimos de {game_name}</b>: {minimun_req}', styles['Normal']))
                                    story_gr.append(Paragraph('<br/><br/>', styles['Normal']))  
                                    story_gr.append(Paragraph(f'<b>Requisitos recomendados de {game_name}</b>: {recommended_req}', styles['Normal']))
                                    story_gr.append(Paragraph('<br/><br/>', styles['Normal']))  

                                    story_gde.append(Paragraph(f'<b>Generos de {game_name}</b>:', styles['Normal']))              
                                    if 'genres' in data:
                                        for genre in data['genres']: story_gde.append(Paragraph(genre['description'], styles['Normal']))
                                    else: story_gde.append(Paragraph('No especificado', styles['Normal']))

                                    story_gde.append(Paragraph(f'<b>Desarrollador de {game_name}</b>: {developers}', styles['Normal']))
                                    story_gde.append(Paragraph('<br/>', styles['Normal']))  
                                    story_gde.append(Paragraph(f'<b>Distribuidor de {game_name}</b>: {publishers}', styles['Normal']))
                                    story_gde.append(Paragraph('<br/><br/>', styles['Normal']))  

                                    writer.writerow([game_name, price])

                    print(f'Progreso: {((games_seen / total_games) * 100):.3f}%.     ', end='\r')

                # Si la respuesta del request nos devuelve el codigo 429 significa que llegamos el limite de la API
                # Solo se pueden hacer 200 llamadas cada 5 minutos, por lo que se espera 5 minutos y se sigue
                elif response.status_code == 429: 
                    print(f'Esperando 5 minutos...', end='\r')

                    games_seen -= 1
                    games.append(game) # Devolvemos el juego ya que este no se analizo

                    time.sleep(300)
                else: print(f'Progreso: {((games_seen / total_games) * 100):.3f}%.', end='\r')
            except JSONDecodeError: pass
            except requests.exceptions.RequestException as e: pass
            finally: pass
    
    # Construimos los documentos PDF con la información recopilada
    game_descriptions.build(story_gd)
    game_requirements.build(story_gr)
    game_details.build(story_gde)               
    
    # Calculamos el tiempo total transcurrido
    end = time.time()
    duracion_segundos = end - start
    horas = duracion_segundos // 3600
    minutos = (duracion_segundos % 3600) // 60
    segundos = duracion_segundos % 60
    
    print(f'PDFs y CSV creados al completo. Tiempo transcurrido: {int(horas)}:{int(minutos)}:{int(segundos)}', end='\r')

## 4.2. *Funciones para Manejo de Datos PDF*

La etapa de limpieza y preparación del texto extraído de los PDFs es fundamental para asegurar que la información esté en un formato adecuado y listo para ser utilizado por el ChatBot. Este proceso se realiza en dos fases principales: limpieza y división del texto.

### Limpieza del texto:
La limpieza del texto implica varios pasos diseñados para eliminar caracteres no deseados y preparar el texto para su posterior procesamiento:

1. **Eliminación de Caracteres Específicos**: Se remueven caracteres que no aportan valor y que resultan de la extracción de los PDFs, como *\x0c* y *[/]*. Estos caracteres suelen ser artefactos de la conversión de formato y no forman parte del contenido útil.

2. **Manejo de Saltos de Línea**: Los dobles saltos de línea dentro de los documentos indican la separación entre secciones y son clave para la estructura del documento. Para preservar esta estructuración, se reemplazan temporalmente con un marcador específico. Esto permite diferenciarlos de los saltos de línea individuales, que sí se desean eliminar o reemplazar, manteniendo así la distinción entre los espacios dentro de un sección y los que delimitan distintos secciones. Luego de ello se vuelve a convertir el marcador en los dos saltos de línea consecutivos.

3. **Limpieza de Espacios en Blanco**: Se eliminan espacios en blanco innecesarios, como múltiples espacios seguidos o espacios al inicio y final de los párrafos, para homogeneizar el texto y facilitar su procesamiento.

### División del Texto:
Una vez el texto ha sido limpiado, el siguiente paso es dividirlo en partes más manejables, basándose en la estructura identificada durante la fase de limpieza:

1. **Uso de Marcadores de División**: Ahora se puede utilizar los dos saltos de líneas consecutivos para dividir el texto en secciones de manera eficiente. Este método asegura que la división se base en la estructura original del documento, respetando la separación intencionada entre secciones.

2. **Identificación de Textos de Inicio de Párrafo**: Gracias a la preparación y formato personalizado de las fuentes de conocimiento, es posible identificar patrones o textos que denoten el comienzo de nuevos párrafos o secciones. Utilizando estos indicadores, junto con los marcadores de división, se facilita aún más la segmentación precisa del texto.

Este proceso de limpieza y división es esencial para transformar el contenido extraído de los PDFs en un recurso listo para ser consultado y analizado por el ChatBot. Al garantizar que el texto esté libre de elementos superfluos y estructurado de manera coherente, se facilita la extracción de información relevante y la generación de respuestas precisas a las consultas de los usuarios.

In [4]:
''' Definimos una función para cargar los documentos PDF desde la ruta especificada '''
def load_pdf_documents(pdf_path):
    documents = []
    for filename in os.listdir(pdf_path):
        if filename.lower().endswith('.pdf'):
            file_path = os.path.join(pdf_path, filename)
            text = extract_text(file_path)
            documents.append(text)
    return documents

''' Definimos una función para limpiar los documentos del texto extraído de los PDFs '''
def clean_documents(documents):
    cleaned_documents = [] # Creamos una lista vacía para almacenar los documentos limpios
    
    # Iteramos sobre cada documento
    for doc in documents:
        # Eliminamos el símbolo '\x0c'
        clean_text = doc.replace('\x0c', '')
        
        # Sustituimos el símbolo [/] por espacio
        clean_text = clean_text.replace('[/]', ' ')
        
        # Conservamos los dobles saltos de línea reemplazándolos temporalmente por un marcador
        clean_text = re.sub(r'(\n\n)', 'DOUBLENEWLINE', clean_text)
        
        # Eliminamos los saltos de línea individuales
        clean_text = re.sub(r'\n', ' ', clean_text)
        
        # Revertimos los dobles saltos de línea a su forma original
        clean_text = clean_text.replace('DOUBLENEWLINE', '\n\n')
        
        # Eliminamos espacios múltiples
        clean_text = re.sub(r' +', ' ', clean_text)
        
        # Lo añadimos a la lista de documentos limpios
        cleaned_documents.append(clean_text)
    return cleaned_documents

''' Definimos una función para dividir el texto de los documentos en segmentos más pequeños '''
def split_text(documents):
    '''
    Condiciones para hacer split:
        1. Que haya dos saltos de líneas iniciales, estos representan un párrafo.
        2. Que después de esos dos saltos de líneas le siga alguna de las siguientes combinaciones de palabras:
                A. 'Descripción corta de'
                B. 'Requisitos mínimos'
                C. 'Requisitos recomendados'
           
           Esto es para que no se haga split en algún lugar donde haya dos saltos de líneas cumpliendo la primera condición
           pero esos saltos uno es porque el texto sigue en una página nueva, por lo que estaríamos cortando texto nuevo.
    '''
    
    # Patrones para hacer split según las condiciones mencionadas
    patrones = r'\n\n(?=Descripción corta de|Requisitos mínimos|Requisitos recomendados)'
    
    # Dividimos el texto usando los patrones definidos
    texts = [text for doc in documents for text in re.split(patrones, doc.strip()) if text]

    # Creamos objetos 'Document' para cada segmento de texto
    chunks = [Document(page_content=text) for text in texts]
    
    print(f'Se dividió {len(documents)} documentos en {len(chunks)} chunks.')
    return chunks
   
''' Definimos una función para guardar los segmentos de texto en una base de datos CHROMA. '''
def save_to_chroma(chunks, model):
    # Cargamos Universal Sentence Encoder
    embed = hub.load(model)
    
    # Configuración inicial de ChromaDB
    client = chromadb.Client()
    
    # Nombre de la colección
    collection_name = 'my-document-collection'
    
    # Eliminamos la colección si ya existe
    for col in client.list_collections():
        if collection_name in col.name: client.delete_collection(collection_name)
    
    # Creamos una nueva colección
    collection = client.create_collection(collection_name)
    
    # Guardamos los 'chunks'
    textos = [doc.page_content for doc in chunks]
    ids_textos = [f"doc{i}" for i in range(1, len(textos) + 1)]
    
    # Calculamos los embeddings para los documentos
    embeddings = embed(textos).numpy().tolist()  # Convertimos a lista para que sea serializable
    
    # Almacenamos los documentos en ChromaDB
    collection.add(
        documents=textos,
        ids=ids_textos,
        embeddings=embeddings
    )
    print(f'Se guardaron {len(chunks)} chunks.')
    return embed, collection

## 4.3. *Funciones para Manejo de Datos CSV*

En el manejo de Datos Tabulares, se procede a eliminar los signos de dólar estadounidense y sus símbolos asociados para conservar únicamente los valores numéricos. Además, se sustituyen los títulos cuyo etiqueta es *Gratis* por el número 0, con el fin de poder manipular los datos de forma más precisa.

In [5]:
''' Definimos una función para convertir los precios de formato texto a formato numérico '''
def convert_price(price):
    if price == 'Gratis' or price == 'Gratuito': return 0.0 # Si el precio es 'Gratis', lo convertimos a 0.0
    # Sino, eliminamos el símbolo del dólar y la especificación de la moneda y convertimos el resultado a un número flotante
    else: return float(price.replace('$', '').replace(' USD', '')) 

''' Definimos una función para cargar y limpiar los datos de un archivo CSV '''
def load_csv():
    data = pd.read_csv(CSV_PATH) # Cargamos los datos desde la ruta al archivo CSV especificado
    data['precio'] = data['precio'].apply(convert_price) # Aplicamos la función 'convert_price' a la columna 'precio'
    return data

## 4.4. *Funciones para la creación y consulta de Base de Datos de Grafos*

En esta sección, procedemos a construir la **Base de Datos de Grafos** utilizando el texto obtenido del PDF que incluye datos relevantes sobre los *géneros*, *desarrolladores* y *distribuidores* de diversos títulos. Opté por crear personalmente la **Base de Datos de Grafos** en lugar de emplear una disponible en internet debido a la similitud en el método de acceso a la información a través de un nodo, comparado con el manejo que realicé con los datos tabulares. Este proceso implica extraer palabras clave de las consultas y utilizar la **Base de datos de Grafos** como de un diccionario se tratase, lo cual resultó ser más sencillo que intentar generar este consultas en SPARQL a partir de la consulta hecha por el usuario, tarea en la que no tuve éxito.

In [15]:
''' Definimos una función para extraer los datos de los PDFs '''
def extract_data(pdf_path):
    data = []

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()

            if text:
                # Encontramos los juegos y la información correspodiente 
                info = re.findall(r'Generos de (.*?):(.*?)Desarrollador de (.*?):(.*?)Distribuidor de (.*?):(.*?)(?=\nGeneros de|\Z)', text, re.DOTALL)

                for game in info:
                    name = game[0].strip()
                    gender = game[1].strip().split(',')
                    developer = game[3].strip()
                    publisher = game[5].strip()

                    data.append({
                        'name': name,
                        'gender': gender,
                        'developer': developer,
                        'publisher': publisher
                    })
    return data

''' Definimos una función para crear la Base de Datos de Grafos con los datos extraidos '''
def create_graph_db():
    data = extract_data("databases/graphDB/Games Detail.pdf") # Extraemos los datos
    
    # Creamos el grafo
    G = nx.Graph()
    
    # Agregamos nodos y aristas, asegurando que los identificadores no estén vacíos
    for game in data:
        if game['name']:  # Aseguramos que el nombre del juego no esté vacío
            G.add_node(game['name'], type='game')
        
        if game['developer']:  # Aseguramos que el nombre del desarrollador no esté vacío
            if game['developer'] == game['publisher'] and game['developer']:
                G.add_node(game['developer'], type='developer/publisher')
                G.add_edge(game['name'], game['developer'])
            else:
                if game['developer']:  # Aseguramos que el nombre del desarrollador no esté vacío
                    G.add_node(game['developer'], type='developer')
                    G.add_edge(game['name'], game['developer'])
                if game['publisher']:  # Aseguramos que el nombre del distribuidor no esté vacío
                    G.add_node(game['publisher'], type='publisher')
                    G.add_edge(game['name'], game['publisher'])
    
        for gender in game['gender']:
            if gender:  # Aseguramos que el género no esté vacío
                G.add_node(gender, type='gender')
                G.add_edge(game['name'], gender)
    return G

''' Definimos funciones para consultar la Base de Datos de Grafos '''
def query_developer(G, extracted_name):
    return next((n for n in G.neighbors(extracted_name) if G.nodes[n]['type'] == 'developer/publisher' or G.nodes[n]['tipo'] == 'developer'), None)

def query_gender(G, extracted_name):
    return [n for n in G.neighbors(extracted_name) if G.nodes[n]['type'] == 'gender']

def query_publisher(G, extracted_name):
    return next((n for n in G.neighbors(extracted_name) if G.nodes[n]['type'] == 'developer/publisher' or G.nodes[n]['tipo'] == 'publisher'), None)

''' Definimos una función para extraer el nombre del juego que se consulta '''
def extract_name(search_string, games, similarity_threshold=80):
    # Utilizamos fuzzywuzzy para encontrar la coincidencia más cercana
    match, similarity = process.extractOne(search_string, games)
    
    # Verificamos si la similitud supera el umbral
    if similarity >= similarity_threshold: return match
    else: return None

''' Definimos una función para extraer el tipo de consulta que se esta haciendo '''
def extract_query_type(query):
    # Palabras clave y algunas variaciones
    keywords = {
        'developer': ['desarrollador', 'desarrolladores', 'creador', 'creadores', 'hizo', 'realizado por'],
        'gender': ['género', 'géneros', 'tipo', 'estilo'],
        'publisher': ['distribuidor', 'distribuidores', 'publicador', 'publicadores', 'lanzado por', 'publicó']
    }

    # Aplananamos la lista de variaciones
    variations = [(key, variation) for key, sublist in keywords.items() for variation in sublist]
    
    # Buscamos la coincidencia más cercana
    best_match = process.extractOne(query, [variation for _, variation in variations], score_cutoff=60)
    
    # Verificamos en caso de no encontrar una coincidencia adecuada
    if best_match:
        match, similarity = best_match
        query_type = next((key for key, variation in variations if variation == match), 'No se encontró coincidencia')
    else: query_type = 'No se encontró coincidencia'
    return query_type

## 4.5. *Funciones para Clasificación de Intenciones*

En esta fase del proyecto, se emplearán los primeros 900 títulos recopilados para entrenar un LLM con el objetivo de predecir a qué base de datos dirigirse basándose en las consultas realizadas por los usuarios. El modelo seleccionado para actuar como clasificador es **BETO: Spanish BERT**, siguiendo la metodología sugerida en la teoría.

Posteriormente, las preguntas se dividen en un conjunto de entrenamiento e pruebas. Se transforman en vectores mediante técnicas de *embeddings*, preparando así el terreno para el entrenamiento del modelo. Este paso permite al modelo clasificar y predecir de manera efecti pero no necesariamente 100% exactava a qué base de datos consultar según la solicitud específica del usuario.

In [7]:
''' Definimos una función para etiquetar y crear preguntas parecidas a las que puede hacer el usuario '''
def collection_labeling(data):
    game_names = data['nombre'].unique() # Obtenemos los nombres únicos de los juegos del CSV  
    dataset = [] # Inicializamos una lista vacía para almacenar las preguntas y etiquetas
    
    # Iteramos sobre los nombres de los juegos, limitando a los primeros 900 juegos para poder usar el resto como validación
    for game in game_names[:900]: 
        # Añadimos preguntas etiquetadas con '0'.
        dataset.append((0, f'¿Cuál sería la descripción corta de {game}?'))
        dataset.append((0, f'¿De qué trata {game}?'))
        dataset.append((0, f'¿Qué se puede hacer en {game}?'))

        dataset.append((0, f'¿Cuáles son los requisitos mínimos de {game}?'))
        dataset.append((0, f'¿Cuáles son los requisitos máximos de {game}?'))
        dataset.append((0, f'¿Qué requisitos necesito para jugar {game}?'))
        dataset.append((0, f'¿Qué requisitos necesito cumplir para jugar {game}?'))
        dataset.append((0, f'¿Qué requisitos necesita mi pc para poder jugar {game}?'))

        # Añadimos preguntas relacionadas etiquetadas con '1'.
        dataset.append((1, f'¿A qué precio está el juego {game}?'))
        dataset.append((1, f'¿Cuál es el precio del juego {game}?'))
        dataset.append((1, f'¿Cuál es el precio de {game}?'))
        dataset.append((1, f'¿Cuánto cuesta {game}?'))
        dataset.append((1, f'¿Cuánto cuesta el juego {game}?'))
        dataset.append((1, f'¿Cuánto vale el juego {game}?'))
        dataset.append((1, f'¿Cuánto vale {game}?'))
        dataset.append((1, f'¿Cuál es el valor de {game}?'))
        dataset.append((1, f'¿Cuál es el valor del juego {game}?'))

        # Añadimos preguntas etiquetadas con '2'.
        dataset.append((2, f'¿Quién es el desarrollador de {game}?'))
        dataset.append((2, f'¿Quién desarrolló {game}?'))
        dataset.append((2, f'¿Quién desarrolló el {game}?'))
        dataset.append((2, f'¿Quién hizo el {game}?'))
        
        dataset.append((2, f'¿Quién publicó el {game}?'))
        dataset.append((2, f'¿Quién distribuyó el {game}?'))
        dataset.append((2, f'¿Quién es el distribuidor del juego {game}?'))
        dataset.append((2, f'¿Quién es el distribuidor de {game}?'))

        dataset.append((2, f'¿Qué genero tiene el juego {game}?'))
        dataset.append((2, f'¿Qué género es {game}?'))
        dataset.append((2, f'¿Qué genero es el juego {game}?'))
        dataset.append((2, f'¿Cuáles generos tiene el juego {game}?'))
        dataset.append((2, f'¿Cuáles son los géneros de {game}?'))
    
    print(f'Se hicieron un total de {len(dataset)} preguntas.')    
    return dataset

'''  Definimos una función para cargar el modelo de embedding BERT. '''
def load_embedding_model():
    model_name = 'dccuchile/bert-base-spanish-wwm-cased' # Definimos el nombre del modelo BERT en español
    tokenizer = BertTokenizer.from_pretrained(model_name) # Cargamos el tokenizador de BERT
    model = BertModel.from_pretrained(model_name) # Cargamos el modelo BERT
    return tokenizer, model

''' Definimos una función para dividir el texto en conjuntos de entrenamiento y prueba '''
def text_split(dataset):
    # Separamos los textos y las etiquetas del conjunto de datos
    X = [text.lower() for label, text in dataset] 
    y = [label for label, text in dataset]
    
    # Dividimos los datos en conjuntos de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)
    return X_train, X_test, y_train, y_test

''' Definimos una función para obtener embeddings de BERT para una lista de textos '''
def get_bert_embeddings(texts, tokenizer, model, progress):
    embeddings = [] # Inicializamos una lista para almacenar los embeddings
    texts_seen = 0
    
    # Iteramos sobre cada texto en la lista
    for text in texts:
        texts_seen += 1
        
        # Preparamos el texto para el modelo BERT utilizando el tokenizador
        inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
        
        # Obtenemos los embeddings del texto sin calcular gradientes
        with torch.no_grad():
            outputs = model(**inputs)
        
        # Añadimos el embedding del primer token a la lista de embeddings
        embeddings.append(outputs.last_hidden_state[0][0].numpy())
        
        if progress: print(f'Progreso: {((texts_seen / len(texts)) * 100):.2f}%     ', end='\r')
    return np.array(embeddings)

## 4.6. *Funciones para Generar Respuesta del Chatbot*

En esta sección, me enfoco en seleccionar la información específica que se proporcionará como contexto al **Chatbot**, basándonos en la fuente de donde necesitemos extraer los datos de acuerdo con la pregunta formulada por el usuario. Dado que este proceso depende de predicciones, es posible que al **Chatbot** se le solicite información que podría considerar irrelevante para la consulta, en caso de acceder a una fuente que no dispone de los datos requeridos. Asimismo, se destaca la elección del modelo de lenguaje utilizado para generar las respuestas, el **Zephyr 7B β**. 

Después de probar varios modelos, este resultó ser mi preferido por la calidad de sus respuestas, siendo mi elección el resultado de un proceso de prueba y error, o más bien, de prueba y precisión.


In [56]:
''' Definimos una función para clasificar una consulta '''
def query_classification(query_text, progress):
    # Convertimos el texto de la consulta en una lista y lo pasamos a minúsculas
    query_text = [query_text] 
    query_text_lower = [text.lower() for text in query_text]
    
    # Obtenemos los embeddings de BERT para el texto de la consulta
    query_text_vectorized = get_bert_embeddings(query_text_lower, tokenizer, model, progress)
    
    # Utilizamos el modelo de regresión logística para predecir la intención de la consulta
    prediction = modelo_LR.predict(query_text_vectorized)
    return prediction[0]

''' Definimos una función para extraer el nombre de un juego de una consulta '''
def extract_name(search_string, games, similarity_threshold=80):
    # Utilizamos fuzzywuzzy para encontrar la coincidencia más cercana
    match, similarity = process.extractOne(search_string, [game for game in games])

    # Verificamos si la similitud supera el umbral
    if similarity >= similarity_threshold: return match
    else: return 'No se encontro información al respecto'

''' Definimos una función para buscar en un CSV '''
def CSV_search(query, df):
    name = extract_name(query, df['nombre'].tolist()) # Extraemos el nombre del juego de la consulta
    
    if name != 'No se encontro información al respecto':
        result = df[df['nombre'] == name] # Buscamos el juego en el DataFrame por su nombre
    
        # Si encontramos el juego, devolvemos su precio
        price = result.iloc[0]['precio']
        return f'El precio de {name} es {str(price)} USD'
    else: return 'No se encontro información al respecto'
        
''' Definimos una función para buscar en documentos PDF '''
def PDF_search(query_text, embed, collection):
    embedding_consulta = embed([query_text]).numpy().tolist()
    results = collection.query(
        query_embeddings=embedding_consulta,
        n_results=3  # Traemos los 3 resultados más cercanos
    )
    
    # Recopilamos los textos de los documentos encontrados y los unimos
    context_text = '\n\n---\n\n'.join([doc for doc in results["documents"][0]])
    return context_text

''' Definimos una función para devolver la información, si se encuentra, correspondiente a la consulta en la BD de Grafos '''
def GRAPH_search(query, G, similarity_threshold=80):
    # Extraemos los nombres de juegos del grafo para la comparación
    games = [n for n, d in G.nodes(data=True) if d.get('type') == 'game']
    query_type = extract_query_type(query)  # Extraemos el tipo de consulta: 'developer', 'gender' o 'publisher'

    # Extraemos el nombre del juego de la consulta
    extracted_name = extract_name(query, games)

    # Llamamos la función correspondiente dependiendo el tipo de consulta
    if extracted_name:
        if query_type == 'developer': 
            developer = query_developer(G, extracted_name)
            query = f'El desarrollador de {extracted_name} es: {developer}'
       
        elif query_type == 'gender': 
            genders_list = query_gender(G, extracted_name)
            genders = ''
            for gender in genders_list: genders += f'{gender} '
            query = f'Los géneros de {extracted_name} son: \n{genders}' 
            query = query.replace('\n', ' ', 1) # Reemplazamos el primer "\n' por un espacio
            query = query.replace('\n', ', ') # Reemplazamos los restantes '\n' por comas
        
        elif query_type == 'publisher': 
            publisher = query_publisher(G, extracted_name)
            query = f'El desarrollador de {extracted_name} es: {publisher}'
            
    else: query = 'No se encontró información relevante.'
    return query

''' Definimos una función para generar un prompt basado en la intención de la consulta '''
def prompt_generate(query_text, intention, embed, collection, data, G):
    # Dependiendo de la intención, buscamos en los documentos PDF o en el CSV
    if intention == 0: context_text = PDF_search(query_text, embed, collection)
    elif intention == 1: context_text = CSV_search(query_text, data)
    elif intention == 2: context_text = GRAPH_search(query_text, G)

    # Formateamos el prompt utilizando la plantilla y los datos obtenidos
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)
    return prompt

''' Definimos una función para generar una respuesta usando la API del modelo LLM '''
def generate_answer(prompt: str, max_new_tokens: int = 768) -> None:
    try:
        # Configuramos la API con la clave y la URL
        api_key = API_KEY  # Ingresamos nuestro API KEY de HugginFace
        api_url = 'https://api-inference.huggingface.co/models/HuggingFaceH4/zephyr-7b-beta'
        headers = {'Authorization': f'Bearer {api_key}'}
        
        # Preparamos los datos para enviar a la API
        data = {
            'inputs': prompt,
            'parameters': {
                'max_new_tokens': max_new_tokens,
                'temperature': 0.7,
                'top_k': 50,
                'top_p': 0.95
            }
        }

        # Enviamos una solicitud POST a la API y obtenemos la respuesta
        response = requests.post(api_url, headers=headers, json=data)
        generated_text = response.json()[0]['generated_text']
        
        # Extraemos la respuesta del texto generado y la imprimimos
        split_text = generated_text.split('Respuesta:') 
        answer = split_text[1].strip().split('\n')[0]

        return print(f'Respuesta: {answer}')
    except Exception as e: print(f'Un error a ocurrido: {e}') # En caso de un error, imprimimos el mensaje de error

''' Definimos una función principal para el chatbot '''
def chatbot(embed, colecction, data, G):
    while True:
        clear_output(wait=True)
        query = input('Ingrese su consulta: ')
    
        # Verificar si la consulta no está vacía
        if query:
            # Asegurar que la consulta comienza con "¿" y termina con "?"
            if not query.startswith('¿'): query = '¿' + query
            if not query.endswith('?'): query = query + '?'
            break
        else: print('Ingrese una consulta válida.')
            
    clear_output(wait=True)
    print(f'Consulta procesada: {query}\n')
    
    prediction = query_classification(query, False) # Clasificamos la consulta para determinar su intención
    prompt = prompt_generate(query, prediction, embed, colecction, data, G) # Generamos un prompt basado en la consulta y su intención
    answer = generate_answer(prompt) # Generamos una respuesta basada en el prompt y la devolvemos
    return answer

# 5. **Creación de Fuentes de Información**

Desde esta sección hasta abajo solo se ejecutan las funciones que se crearon con anterioridad junto a una desmotración paso a paso sobre que hace el **ChatBot**. Al final también hay una DEMO por si se quiere consultar directamente al **Chatbot** con varias consultas consecutivas para ir evaluando su funcionamiento.

In [ ]:
# Traemos los nombres de los juegos
games = bring_game_names()

# Creamos los PDFs y CSV con la información necesaria para el chatbot
game_details(games[:10000]) # Lo limitamos a 10.000 porque sino tardaría muchisimo tiempo en traer todos (+180.000)

# 6. **Carga y Limpieza de Datos**

## 6.1. *Documentos PDF*

In [9]:
# Cargamos los documentos PDF
raw_documents = load_pdf_documents(PDF_PATH)

# Limpiamos los documentos
documents = clean_documents(raw_documents)

## 6.2. *Datos Tabulares (CSV)*

In [10]:
# Cargamos y limpiamos el csv
data = load_csv()

## 6.3 *Recolección y Etiquetado de Preguntas para Clasificación de Intenciones*

In [11]:
# Creamos las preguntas y sus etiquetas
dataset_intentions = collection_labeling(data)

Se hicieron un total de 26490 preguntas.


# 7. **Procesamiento de Datos para Base de Datos Chroma (PDFs)**

## 7.1. *División de los Documentos en Pequeñas Partes*

In [12]:
# Dividimos los documentos en pequeñas partes
chunks = split_text(documents)

Se dividió 2 documentos en 2649 chunks.


## 7.2. *Conversión a Embeddings y Almacenamiento en Chroma*

In [13]:
# Convertimos las partes utilizando 'Embeddings' y las almacenamos en una base de datos'Chroma'
embed, collection = save_to_chroma(chunks, model_embedding_chatbot)

Se guardaron 2649 chunks.


# 8. **Creación de la Base de Datos de Grafos**

In [16]:
# Creamos la Base de Datos de Grafos
G = create_graph_db()

# 9. **Entrenamiento del Clasificador de Intenciones**

## 9.1. *Cargamos el Modelo de Embeddings*

In [17]:
# Cargamos el modelo
tokenizer, model = load_embedding_model()

Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 9.2. *División en Conjunto de Entrenamiento y Prueba y Tokenización*

In [18]:
# Dividimos nuestro Dataset
X_train, X_test, y_train, y_test = text_split(dataset_intentions)

# Obtenemos los embeddings de BERT para los conjuntos de entrenamiento y prueba
X_train_vectorized = get_bert_embeddings(X_train, tokenizer, model, True)
X_test_vectorized = get_bert_embeddings(X_test, tokenizer, model, True)

## 9.3. *Entrenamiento y Evaluación del Modelo Clasificador*

In [19]:
# Creamos y entrenamamos el modelo de Regresión Logística Multinomial
modelo_LR = LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs')
modelo_LR.fit(X_train_vectorized, y_train)

# Evaluamos el modelo de Regresión Logística
y_pred_LR = modelo_LR.predict(X_test_vectorized)
acc_LR = accuracy_score(y_test, y_pred_LR)
report_LR = classification_report(y_test, y_pred_LR, zero_division=1)

print('Precisión Regresión Logística:', acc_LR)
print('Reporte de clasificación Regresión Logística:\n', report_LR)

Precisión Regresión Logística: 0.9998112495281238
Reporte de clasificación Regresión Logística:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      1402
           1       1.00      1.00      1.00      1622
           2       1.00      1.00      1.00      2274

    accuracy                           1.00      5298
   macro avg       1.00      1.00      1.00      5298
weighted avg       1.00      1.00      1.00      5298



## 9.4. *Validación del Modelo Clasificador*

In [20]:
# Nuevas frases para clasificar como validación del modelo
new_phrases = [
    '¿Cuanto vale Traffix?',
    '¿De qué se trata Mystery Island - Hidden Object Games?',
    '¿Cuáles son los requisitos de Nyanco?',
    '¿Quién desarollo de Pushy and Pully in Blockland?',
    '¿Cuanto valdria DOTORI?',
    '¿Quién desarollo Escape Dead Earth?',
    '¿Cuáles son los géneros de DinosaurIsland?'
]

# Preprocesamiento y vectorización de las nuevas frases
new_phrases_lower = [text.lower() for text in new_phrases]
new_phrases_vectorized = get_bert_embeddings(new_phrases_lower, tokenizer, model, False)

# Hacemos predicciones con el modelo entrenado
new_predictions = modelo_LR.predict(new_phrases_vectorized)

# Mostramos las predicciones junto con las frases
for text, label in zip(new_phrases, new_predictions):
    print(f'Texto: "{text}"')
    print(f'Clasificación predicha: {labels[label][1]}\n')

Texto: "¿Cuanto vale Traffix?"
Clasificación predicha: Consulta CSV

Texto: "¿De qué se trata Mystery Island - Hidden Object Games?"
Clasificación predicha: Consulta PDF

Texto: "¿Cuáles son los requisitos de Nyanco?"
Clasificación predicha: Consulta PDF

Texto: "¿Quién desarollo de Pushy and Pully in Blockland?"
Clasificación predicha: Consulta Grafos

Texto: "¿Cuanto valdria DOTORI?"
Clasificación predicha: Consulta CSV

Texto: "¿Quién desarollo Escape Dead Earth?"
Clasificación predicha: Consulta Grafos

Texto: "¿Cuáles son los géneros de DinosaurIsland?"
Clasificación predicha: Consulta Grafos



# 10. **Integración y Flujo de Trabajo del Chatbot**

## 10.1. Definimos la consulta a hacer

In [52]:
# Pregunta hecha por el 'usuario'
query = "¿Cuál es la descripción de Game Developer Simulator?"

## 10.2. *Clasificación de Pregunta de Usuario*

In [53]:
# Clasificamos la query para saber que base de datos usar
prediction = query_classification(query, False)

## 10.3. *Búsqueda en la Base de Datos Correspondiente y Generación del Prompt*

In [54]:
# Buscamos el contexto correspondiente y generamos el prompt
prompt = prompt_generate(query, prediction, embed, collection, data, G)

## 10.4. *Generación de Respuesta con el Chatbot*

In [57]:
# Generamos la respuesta del Chatbot
answer = generate_answer(prompt)

Respuesta: En el Simulador de un desarrollador de juegos, se puede comprender todo lo subtil de esta profesión, comprendar cómo funciona la industria de los juegos y todas las dificultades de un solo desarrollador.


# 11. **Demo del Chatbot**

In [59]:
chatbot(embed, collection, data, G)

Consulta procesada: ¿Cuál es la descripción de Developer Simulator?

Respuesta: La descripción corta de Developer Simulator es: "En el Simulador de un desarrollador de juegos, podrá comprender todas las sutilezas de esta profesión, comprender cómo funciona la industria de los juegos y todas las dificultades de un solo desarrollador."


# 12. **Informe sobre Agentes Inteligentes (LLM libres)**

## 12.1. *Estado del Arte de las Aplicaciones Actuales*

La evolución de los agentes inteligentes y los modelos LLM ha revolucionado diversas áreas, desde el análisis de datos hasta la interacción en lenguaje natural. Plataformas como *OpenAgents* buscan democratizar el acceso a estas tecnologías, ofreciendo una base para la creación y el uso de agentes inteligentes basados en LLM de manera más abierta y accesible. 

Estos agentes pueden especializarse en tareas como el análisis de datos, la integración con **APIs** para tareas cotidianas y la navegación web anónima. La interacción con estos agentes se facilita a través de interfaces web, permitiendo a los usuarios, desarrolladores e investigadores colaborar y expandir sus funcionalidades.

Además, el impacto de los modelos **LLM** se ha notado en la mejora de la interacción usuario-máquina, permitiendo a cualquier persona, sin necesidad de conocimientos técnicos, interactuar satisfactoriamente con estos sistemas. Los agentes inteligentes se han vuelto parte de la vida cotidiana, ofreciendo desde asistencia virtual hasta sugerencias de productos personalizadas, lo que demuestra su capacidad de aprendizaje y adaptación.

## 12.2. *Problemática a Solucionar con un Sistema Multiagente*

Una problemática relevante en la sociedad actual es la gestión eficiente de las respuestas ante emergencias, como desastres naturales o accidentes urbanos. La coordinación rápida y efectiva entre los diferentes servicios de emergencia puede ser la diferencia entre salvar vidas y mitigar daños.

El sistema multiagente propuesto incluiría los siguientes agentes, cada uno especializado en una tarea crítica de la gestión de emergencias:
* **Agente de Detección**: Utilizaría tecnologías de procesamiento de lenguaje natural y análisis de datos para monitorear redes sociales, noticias y sensores IoT (sensores inteligentes) para detectar posibles emergencias en tiempo real.

* **Agente de Coordinación**: Este agente, basado en modelos de toma de decisiones y optimización, sería el encargado de evaluar la información recopilada y coordinar la respuesta entre los diferentes servicios de emergencia

* **Agente de Logística**: Especializado en la optimización de rutas y recursos, este agente garantizaría la asignación eficiente de recursos y personal de emergencia al lugar del incidente.

* **Agente de Comunicación**: Utilizaría LLM para interactuar con el público y los servicios de emergencia, proporcionando información actualizada, recomendaciones y recopilando datos relevantes del terreno.

## 12.3. *Ejemplo de Conversación*

**Agente de Detección**: *"He detectado un aumento significativo de mensajes en redes sociales sobre un posible incendio forestal en la región de la Sierra Norte. Los sensores de temperatura en la zona también muestran lecturas anómalas."*

**Agente de Coordinación**: *"Recibido, Agente de Detección. Solicitando al Agente de Logística que evalúe la disponibilidad y la ubicación de los equipos de bomberos más cercanos y la mejor ruta para llegar al lugar del incendio."*

**Agente de Logística**: *"El equipo de bomberos más cercano está en la estación de San Miguel, a 10 km del incendio. La ruta más rápida está actualmente despejada y tomará aproximadamente 15 minutos. Estoy coordinando la movilización inmediata."*

**Agente de Comunicación**: *"Informando a la población local sobre el incendio. Mensaje enviado: 'Alerta de incendio en la Sierra Norte. Por favor, eviten el área y sigan las instrucciones de evacuación si se encuentran cerca. Manténganse informados para más actualizaciones.' También estoy informando a los equipos de bomberos con detalles sobre la situación y las coordenadas exactas del incendio."*

## 12.4. *Fuentes de Información*

[**Unite.AI**: *OpenAgents: una plataforma abierta para agentes lingüísticos en la naturaleza*](https://www.unite.ai/es/openagents-una-plataforma-abierta-para-agentes-lingüísticos-en-la-naturaleza/)

[**Hiberus**: *Estado del arte de las IAs: la inteligencia artificial en la actualidad*](https://www.hiberus.com/crecemos-contigo/sobre-la-inteligencia-artificial-en-la-actualidad/)

[**UNIR**: *Los agentes inteligentes: Funciones y ejemplos de uso*](https://www.unir.net/ingenieria/revista/agentes-inteligentes/)